In [1]:
import time,requests,json,random
import pandas as pd
import datetime as dt


In [2]:
class StockCrawling:
    req = None
    db = None

    def __init__(self):
        self.req = requests.session()

    def __del__(self):
        self.req.close()

    def show_realtime(self, *stock_id):
        twse_url = 'http://mis.twse.com.tw/stock/api/getStockInfo.jsp'
        timestamp = int(time.time() * 1000)
        channels = '|'.join('tse_{}.tw'.format(target_tse) for target_tse in stock_id)
        query_url = '{}?&ex_ch={}&json=1&delay=0&_={}'.format(twse_url, channels, timestamp)

        headers = {'Accept-Language': 'zh-TW',
                   'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.81 Safari/537.36',
                   }

        self.req.get('http://mis.twse.com.tw/stock/index.jsp', headers=headers)
        response = self.req.get(query_url)
        if response.text.strip() == '':
            return None

        # d: 日期, h: 最高, l: 最低, c:代號, n: 名稱, t:時間, o: 開盤, v: 交易量, z: 成交價
        # load JSON from TSE
        print(response.text)
        content = json.loads(response.text)
        self.req.cookies.clear()
        return content['msgArray']

In [3]:
if __name__ == '__main__':
    # To ill stock IDs that you want.
    data = pd.read_csv("./ListedStockID/其他類.csv")
    stock_ids = [ str(i) for i in data['代碼'].to_list() ]
    crawling = StockCrawling()
    try:
        while True:
            now = dt.datetime.now()
            #if now < STOP_TIME:
            data = crawling.show_realtime(*stock_ids)
            records = []
            for s_dict in data:
                try:
                    _t = dt.datetime.strptime(s_dict['t'], '%H:%M:%S').time()
                    _dt = dt.datetime.combine(dt.datetime.strptime(s_dict['d'], "%Y%m%d").date(), _t).strftime(
                    '%Y-%m-%d %H:%M:%S')
                    records.append([_dt, s_dict['c'], s_dict['n'],
                                float(s_dict['z']), float(s_dict['h']), float(s_dict['l']), float(s_dict['o']),
                                int(s_dict['v'])])
                except ValueError as _e:
                    print("ValueErr : " + str(_e))
            print(records)
            _sleep_time = random.randint(2, 4)
                # sleep
            time.sleep(_sleep_time)
            print("====== " + time.strftime("%H:%M:%S") + " ======")
    finally:
        print("ERROR")
        del crawling


























{"msgArray":[{"tv":"2","ps":"2","pz":"10.7500","bp":"0","fv":"0","oa":"10.8500","ob":"10.7500","a":"10.8500_10.9000_10.9500_11.0000_11.0500_","b":"10.7500_10.7000_10.6500_10.6000_10.5500_","c":"9902","d":"20220824","ch":"9902.tw","ot":"14:30:00","tlong":"1661322600000","f":"9_10_26_20_3_","ip":"0","g":"46_36_16_11_12_","mt":"000000","ov":"500","h":"10.8500","i":"20","it":"12","oz":"10.8500","l":"10.7500","n":"台火","o":"10.8000","p":"0","ex":"tse","s":"2","t":"13:30:00","u":"11.8000","v":"43","w":"9.6800","nf":"台火開發股份有限公司","y":"10.7500","z":"10.7500","ts":"0"},{"tv":"614","ps":"563","pz":"29.0000","bp":"0","fv":"4","oa":"28.9000","ob":"28.8500","a":"28.9500_29.0000_29.0500_29.1000_29.1500_","b":"28.9000_28.8500_28.8000_28.7500_28.7000_","c":"9904","d":"20220824","ch":"9904.tw","ot":"14:30:00","tlong":"1661322600000","f":"26_227_118_122_100_","ip":"0","g":"50_322_287_176_228_","mt":"000000","ov":"1614","h":"29.0500","i":"20","it":"12","oz":"28.9000","l":"28.8500","

KeyboardInterrupt: 